# 要約 
このJupyter Notebookは、Kaggleコンペティション「LMSYS - Chatbot Arena」における人間による好みの予測を行うための機械学習モデルの開発を目的としています。具体的には、異なる大規模言語モデル（LLM）が生成した応答のうち、ユーザーに最も好まれるものを予測するモデルを設計しています。

## 問題の取り組み
このNotebookでは、特に以下の課題に取り組んでいます：
- LLMの応答の中から、どのモデルがユーザーの好みに合致するかを予測する。
- LoRA（Low-Rank Adaptation）を使用したモデルの微調整。
- 最大シーケンス長を768に設定し、メモリ制約を考慮する。

## 使用手法とライブラリ
Notebookは多くのPyTorchとTransformersライブラリのモジュールを活用しています。以下に主な手法とライブラリを示します：

- **データ前処理**: サンプリング、欠損値の除去、ラベルの変換などを行っています。データはCSVファイルから読み込まれ、必要に応じてJSON形式に変換されます。
- **カスタムデータセット**:
  - `CustomDataset`クラスを使用して、トークン化されたプロンプトと応答をデータローダーに組み込みます。
  - `create_dataloaders`関数を介して、訓練と評価用のデータローダーを作成します。
  
- **モデル構築**: 
  - MicrosoftのPhi-3-mini-4k-instructモデルをベースに、LoRAアダプターを追加したカスタムモデルを定義しています。これにより、少ないパラメータでモデルのパフォーマンスを向上させることができます。
  - `Phi3Attention`クラスをカスタマイズして、LoRAアダプターが適用されたマルチヘッダーアテンションを実装しています。

- **トレーニングプロセス**:
  - 半精度トレーニングを行い、モデルのトレーニングと評価をパラレルに処理します。
  - 最適化手法にはAdamWが使用され、学習率スケジューラも設定されています。
  - 評価指標として、精度とF1スコアを計算・表示しています。

- **量子化**: 
  - モデルの重みを量子化する関数が実装されており、モデルのサイズを削減し、効率的な推論を行うことが可能です。

このNotebookでは、最終的にトレーニングしたモデルを保存し、後で利用できるようにしています。このプロセス全体を通じて、特にユーザーの好みに基づく予測を行うための効果的な機械学習手法を実装しています。

---


# 用語概説 
以下に、機械学習・深層学習の初心者がつまずきやすい専門用語を解説します。ノートブック特有のドメイン知識や実務経験が必要な用語に焦点を当てています。

1. **LoRA (Low-Rank Adaptation)**:
   - LoRAは、深層学習モデルのパラメータを効率的に更新するための手法です。この手法では、重み行列を低ランクで分解し、追加のパラメータを訓練することで少ないリソースでモデルの適応を行います。特に大規模モデルの微調整に利用されます。

2. **量子化 (Quantization)**:
   - 量子化とは、モデルのパラメータや計算を低ビット数で表現する技術です。これにより、モデルのサイズが小さくなり、メモリや計算資源の消費が削減されます。特にGPUやTPUなどのハードウェアを効果的に使用する際に有用です。

3. **Attention Mask**:
   - Attention Maskは、自己注意機構がどの入力トークンを考慮するかを示すためのバイナリマスクです。特にパディングのトークンを無視したり、一部のトークンのみを参照したりする場合に使用されます。

4. **アダプタレイヤー (Adapter Layer)**:
   - アダプタレイヤーは、既存のモデルに対して新しい知識を効率的に追加できるように設計された層です。この層を追加することで、モデル全体を再訓練せずに特定のタスクに特化した機能を持たせることができます。

5. **Flash Attention**:
   - Flash Attentionは、メモリ使用量を最適化し、自己注意をより効率的に計算するための手法です。特に長いシーケンスを扱うタスクで有用で、計算速度の向上とメモリ効率の改善を実現します。

6. **Rotary Positional Embedding (RoPE)**:
   - RoPEは、トークン間の位置情報を表現するための手法で、特に長距離依存性を持つシーケンスモデルにおいて有効です。この手法により、位置情報のエンコーディングがより柔軟かつ効率的になります。

7. **Dropout**:
   - Dropoutは、過学習を防ぐための正則化手法で、トレーニング中にランダムに一定割合のニューロンを無効化しますが、ここでは特にLoRA構造内の使用がソフトウェアエンジニアにとって新しい概念であることが考えられます。

8. **Mixed Precision Training**:
   - Mixed Precision Trainingは、モデルのトレーニングにおいて異なる精度（例：16ビット浮動小数点数と32ビット浮動小数点数）を混ぜて使用する手法です。この手法は、GPUメモリの使用効率を向上させ、計算速度を速めるために使用されます。

これらの用語は、機械学習の特定の側面や、特定のライブラリの機能に関連しており、初学者が遭遇することがよくある重要な概念です。

---


# インストール不要

microsoft/Phi-3-mini-4k-instruct + LoRA > GPUでの並列トレーニング

最大シーケンス長はモデルのパフォーマンスに大きな影響を与えますが、メモリ不足のため、最大長は768に設定されました。


## ライブラリの読み込み


In [ ]:
import multiprocessing as mp
mp.set_start_method('spawn')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import datasets
from datasets import load_dataset, load_metric, Dataset

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, log_loss

from accelerate import notebook_launcher, Accelerator, PartialState
from accelerate.utils import write_basic_config
from accelerate.inference import prepare_pippy

import transformers
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
    set_seed,
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    AutoConfig
)

import os
import shutil
import math
import json
from tqdm import tqdm
import gc
import pandas as pd
import numpy as np
from typing import Optional, Tuple

In [ ]:
# パラメータ設定
model_name = "/kaggle/input/microsoftphi-3-mini-4k-instruct/transformers/default/1"  # モデルの名前
model_path = "model_checkpoint.pth"  # モデルのチェックポイントのパス
seed = 42  # 乱数シード
lora_r = 2  # LoRAのランク
quantize_bit = 16  # 量子化ビット数
learning_rate = 5e-4  # 学習率
weight_decay = 0.1  # 重み減衰
beta1 = 0.9  # Adamオプティマイザのβ1
beta2 = 0.999  # Adamオプティマイザのβ2
eps = 1e-9  # 許容誤差
l1_rate = 1e-10  # L1正則化率
batch_size = 1  # バッチサイズ
max_len = 256  # 最大シーケンス長
n_sample = 0.10  # サンプリング比率
n_epoch = 2  # エポック数
device = "cuda"  # 使用するデバイス
file_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'  # トレーニングデータのファイルパス

## 前処理


In [ ]:
def cl(x):
  if x == [1,0,0]:
    return 0  # モデルAが勝者の場合
  elif x == [0,1,0]:
    return 1  # モデルBが勝者の場合
  else:
    return 2  # 引き分けの場合

In [ ]:
def preprocess_data(file_path, sample=False):
    train = pd.read_csv(file_path)  # 訓練データをCSVから読み込む
    clf_train = train[['prompt','response_a','response_b','winner_model_a','winner_model_b','winner_tie']]

    # JSON形式から最初の要素を抽出
    clf_train.loc[:, "prompt"] = clf_train["prompt"].apply(lambda x: json.loads(x)[0])
    clf_train.loc[:, "response_a"] = clf_train["response_a"].apply(lambda x: json.loads(x)[0])
    clf_train.loc[:, "response_b"] = clf_train["response_b"].apply(lambda x: json.loads(x)[0])

    clf_train = clf_train.dropna()  # 欠損値を削除
    clf_train = clf_train.reset_index(drop=True)  # インデックスをリセット

    # 各勝者を含むターゲット列を作成
    clf_train['target'] = [[clf_train['winner_model_a'][x],clf_train['winner_model_b'][x],clf_train['winner_tie'][x]] for x in range(len(clf_train))]

    clf_train = clf_train[['prompt','response_a','response_b','target']]

    # ターゲットをラベルに変換
    clf_train['labels'] = clf_train['target'].apply(lambda x : cl(x))

    # 各要素の長さを計算
    clf_train['p_len'] = clf_train['prompt'].apply(lambda x : len(x))
    clf_train['a_len'] = clf_train['response_a'].apply(lambda x : len(x))
    clf_train['b_len'] = clf_train['response_b'].apply(lambda x : len(x))

    clf_train['len'] = clf_train['p_len'] + clf_train['a_len'] + clf_train['b_len']
    
    if sample:  # サンプリングを行う場合
        clf_train = clf_train.sample(int(len(clf_train) * n_sample), weights="len", random_state=seed).reset_index(drop=True)

    t_dat, v_dat = train_test_split(clf_train, test_size=0.2, random_state=42, stratify=clf_train['labels'])  # データセットを訓練と検証に分割

    t_dat = t_dat.reset_index(drop=True)
    v_dat = v_dat.reset_index(drop=True)

    t_dat = t_dat.drop(labels='target', axis=1)  # 'target'列を削除
    v_dat = v_dat.drop(labels='target', axis=1)  # 'target'列を削除
    return t_dat, v_dat  # 訓練データと検証データを返す

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.prompt = df['prompt']
        self.response_a = df['response_a']
        self.response_b = df['response_b']
        self.max_len = max_len
        self.targets = df.get('labels', None)  # ラベルを取得

    def __len__(self):
        return len(self.prompt)  # プロンプトの数を返す

    def __getitem__(self, index):
        # インデックスに基づいてプロンプトおよび応答を取得
        prompt = str(self.prompt[index])
        response_a = str(self.response_a[index])
        response_b = str(self.response_b[index])

        # 各テキストのトークナイズされた長さを計算
        prompt_len = len(self.tokenizer("##prompt: " + prompt, add_special_tokens=True)['input_ids'])
        response_a_len = len(self.tokenizer("##response_a: " + response_a, add_special_tokens=True)['input_ids'])
        response_b_len = len(self.tokenizer("##response_b: " + response_b, add_special_tokens=True)['input_ids'])

        # 最大長を超えないように長さを制限
        final_prompt_len = min(self.max_len, prompt_len)
        final_a_len = min(self.max_len, response_a_len)
        final_b_len = min(self.max_len, response_b_len)

        # トークナイズ
        prompt_token = self.tokenizer("##prompt: " + prompt, add_special_tokens=True, max_length=final_prompt_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
        response_a_token = self.tokenizer("##response_a: " + response_a, add_special_tokens=True, max_length=final_a_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
        response_b_token = self.tokenizer("##response_b: " + response_b, add_special_tokens=True, max_length=final_b_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')

        # 入力IDとアテンションマスクを結合
        input_ids = torch.cat([prompt_token['input_ids'], response_a_token['input_ids'], response_b_token['input_ids']], dim=1)
        attention_mask = torch.cat([prompt_token['attention_mask'], response_a_token['attention_mask'], response_b_token['attention_mask']], dim=1)

        if self.targets is not None:
            labels = torch.LongTensor([self.targets[index]])
            return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten(), 'labels': labels}  # 入力ID、アテンションマスク、ラベルを返す
        else:
            return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten()}  # 入力IDとアテンションマスクだけを返す

In [ ]:
def custom_collate_fn(batch, tokenizer):

    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = torch.cat([item['labels'] for item in batch], dim=0) if 'labels' in batch[0] else None

    # バッチ内のシーケンスの最大長を見つける
    max_len = max([input_id.size(0) for input_id in input_ids])

    # 新しい最大長で再トークナイズ
    new_input_ids = []
    new_attention_masks = []

    for item in batch:
        input_ids = item['input_ids'][:max_len]  # 最大長で切り取る
        attention_mask = item['attention_mask'][:max_len]  # 最大長で切り取る

        new_input_ids.append(input_ids)  # 新しい入力IDに追加
        new_attention_masks.append(attention_mask)  # 新しいアテンションマスクに追加

    new_input_ids = pad_sequence(new_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)  # パディングを適用
    new_attention_masks = pad_sequence(new_attention_masks, batch_first=True, padding_value=0)  # パディングを適用

    output = {
    'input_ids': new_input_ids,
    'attention_mask': new_attention_masks}

    if labels is not None:
        output['labels'] = labels  # ラベルがある場合は追加

    return output

In [ ]:
def create_dataloaders(df, tokenizer, max_len, batch_size, shuffle=True):
    # データローダーを作成するための関数
    dataloader = DataLoader(
        CustomDataset(df, tokenizer, max_len), shuffle=shuffle, batch_size=batch_size , collate_fn=lambda x: custom_collate_fn(x, tokenizer)
    )
    return dataloader

## モデルの読み込み


In [ ]:
def quantize_tensor(tensor, num_bits=quantize_bit):
    qmin = 0.
    qmax = 2.**num_bits - 1.  # 量子化の最大値を計算

    min_val, max_val = tensor.min(), tensor.max()  # テンソルの最小値と最大値を取得
    scale = (max_val - min_val) / (qmax - qmin)  # スケールを計算
    zero_point = qmin - min_val / scale  # ゼロポイントを計算

    quantized_tensor = torch.round(tensor / scale + zero_point)  # テンソルを量子化
    quantized_tensor = torch.clamp(quantized_tensor, qmin, qmax)  # 最大・最小値でクリップ
    quantized_tensor = (quantized_tensor - zero_point) * scale  # 再スケーリング

    return quantized_tensor

def quantize_model(model, num_bits=8):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):  # 線形層の場合
            module.weight.data = quantize_tensor(module.weight.data, num_bits)  # 重みを量子化
            if module.bias is not None:
                module.bias.data = quantize_tensor(module.bias.data, num_bits)  # バイアスを量子化
        elif isinstance(module, nn.Conv2d):  # 畳み込み層の場合
            module.weight.data = quantize_tensor(module.weight.data, num_bits)  # 重みを量子化
            if module.bias is not None:
                module.bias.data = quantize_tensor(module.bias.data, num_bits)  # バイアスを量子化

    return model


# import torch.quantization

# def quantize_model_dynamic(model):
#     model.qconfig = torch.quantization.default_dynamic_qconfig
#     torch.quantization.prepare(model, inplace=True)
#     torch.quantization.convert(model, inplace=True)
#     return model

## アダプタレイヤーの追加
transformers.models.phi3.modeling_phi3.Phi3Attentionからコピー

[GitHubのURL](https://github.com/huggingface/transformers/blob/main/src/transformers/models/phi3/modeling_phi3.py)


In [ ]:
class LoRA(nn.Module):
    def __init__(self, in_features, out_features, rank=lora_r, alpha=1.0, lora_dropout=0.05):
        super(LoRA, self).__init__()
        self.alpha = alpha  # Alphaパラメータ
        self.rank = rank  # LoRAのランク
        self.lora_a = nn.Linear(in_features, rank, bias=False)  # LoRA A行列
        self.lora_b = nn.Linear(rank, out_features, bias=False)  # LoRA B行列
        self.dropout = nn.Dropout(lora_dropout)  # ドロップアウト層

    def forward(self, x):
        lora_out = self.alpha * self.lora_b(self.lora_a(x))  # LoRAの出力を計算
        lora_out = self.dropout(lora_out)  # ドロップアウトを適用
        return lora_out

In [ ]:
from transformers.models.phi3.modeling_phi3 import (
Phi3RotaryEmbedding, 
# Phi3LongRoPEScaledRotaryEmbedding,
apply_rotary_pos_emb,
repeat_kv
)

In [ ]:
class Phi3Attention(nn.Module):
    """ 'Attention Is All You Need' 論文に基づくマルチヘッダーアテンション """

    def __init__(self, config, layer_idx: Optional[int] = None):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        if layer_idx is None:
            logger.warning_once(
                f"{self.__class__.__name__} を layer_idx を指定せずにインスタンス化するのは推奨されず、"
                "キャッシュを使用している場合にフォワード呼び出し時にエラーが発生する可能性があります。"
                "このクラスを作成する際は、layer_idx を指定してください。"
            )

        # 各種設定を初期化
        self.attention_dropout = config.attention_dropout
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.original_max_position_embeddings = config.original_max_position_embeddings
        self.rope_theta = config.rope_theta
        self.rope_scaling = config.rope_scaling
        self.is_causal = True

        # hidden_size が num_heads で割り切れない場合にエラーを発生させる
        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size は num_heads で割り切れる必要があります (得られた `hidden_size`: {self.hidden_size}"
                f" と `num_heads`: {self.num_heads})。"
            )

        # 線形変換のためのサイズを計算
        op_size = self.num_heads * self.head_dim + 2 * (self.num_key_value_heads * self.head_dim)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)
        self.qkv_proj = nn.Linear(self.hidden_size, op_size, bias=False)
        self._init_rope()
        
        ########################## LoRAアダプタ ##########################
        self.qkv_lora = LoRA(self.hidden_size, op_size, lora_r)  # LoRAアダプタの初期化
        self.o_lora = LoRA(self.num_heads * self.head_dim, self.hidden_size, lora_r)  # LoRAアダプタの初期化
        ########################## LoRAアダプタ ##########################
        
    def _init_rope(self):
        if self.rope_scaling is None:
            self.rotary_emb = Phi3RotaryEmbedding(
                self.head_dim,
                max_position_embeddings=self.max_position_embeddings,
                base=self.rope_theta,
            )
        else:
            scaling_type = self.config.rope_scaling["type"]
            if scaling_type == "longrope":
                self.rotary_emb = Phi3LongRoPEScaledRotaryEmbedding(self.head_dim, self.config)
            else:
                raise ValueError(f"未知の RoPE スケーリングタイプ {scaling_type}")

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value=None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        # logger.warning_once("フラッシュアテンションの実装を実行していないため、数値的な差異が予想されます。")

        bsz, q_len, _ = hidden_states.size()
        ########################## LoRAアダプタ ##########################
        qkv = self.qkv_proj(hidden_states) + self.qkv_lora(hidden_states)  # LoRAアダプタを適用
        ########################## LoRAアダプタ ##########################
        query_pos = self.num_heads * self.head_dim
        query_states = qkv[..., :query_pos]
        key_states = qkv[..., query_pos:query_pos + self.num_key_value_heads * self.head_dim]
        value_states = qkv[..., query_pos + self.num_key_value_heads * self.head_dim:]

        # 各状態の形状を整形
        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

        kv_seq_len = key_states.shape[-2]
        if past_key_value is not None:
            if self.layer_idx is None:
                raise ValueError(
                    f"キャッシュ構造はバージョンv4.36以降変更されました。{self.__class__.__name__} を使用して自動回帰デコーディングを行っている場合、"
                    "レイヤーインデックスで初期化されていることを確認してください。"
                )
            kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
        cos, sin = self.rotary_emb(value_states, position_ids, seq_len=kv_seq_len)

        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

        if past_key_value is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # RoPEモデル専用の引数
            key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

        # k/v ヘッドが n_kv_heads < n_heads の場合は重複
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        value_states = repeat_kv(value_states, self.num_key_value_groups)

        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):
            raise ValueError(
                f"Attention weights は {(bsz, self.num_heads, q_len, kv_seq_len)} サイズであるべきですが、"
                f" {attn_weights.size()} です。"
            )

        if attention_mask is not None:
            causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
            attn_weights += causal_mask

        # atención を fp32 にアップキャスト
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(value_states.dtype)
        attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)

        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"`attn_output` は {(bsz, self.num_heads, q_len, self.head_dim)} サイズであるべきですが、"
                f" {attn_output.size()} です。"
            )

        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)
        ########################## LoRAアダプタ ##########################
        attn_output = self.o_proj(attn_output) + self.o_lora(attn_output)  # LoRAを適用
        ########################## LoRAアダプタ ##########################
        if not output_attentions:
            attn_weights = None

        return attn_output, attn_weights, past_key_value

In [ ]:
def replace_attention_module(config, layer, layer_idx):
    if hasattr(layer, 'self_attn') and layer_idx > 12:

        new_attention = Phi3Attention(config, layer_idx)  # 新しいPhi3Attentionを作成

        # 重みをコピー
        new_attention.qkv_proj.weight.data.copy_(layer.self_attn.qkv_proj.weight.data)
        new_attention.o_proj.weight.data.copy_(layer.self_attn.o_proj.weight.data)

        layer.self_attn = new_attention  # 置き換え

In [ ]:
loss_fn = nn.CrossEntropyLoss()  # クロスエントロピー損失関数の初期化

class LoraModelForClassification(nn.Module):
    def __init__(self, lora_model):  # LoRAモデルを受け取る
        super(LoraModelForClassification, self).__init__()
        self.config = lora_model.config  # 設定を保存
        self.peft_model = lora_model  # LoRAモデルを保存
        self.dropout = nn.Dropout(0.1)  # ドロップアウト層
        self.classifier = nn.Linear(self.config.hidden_size, 3)  # 最終分類層の初期化
#         self.classifier.weight.data = self.classifier.weight.data.to(torch.float16)  # 重みのデータ型を変更
#         self.classifier.bias.data = self.classifier.bias.data.to(torch.float16)  # バイアスのデータ型を変更

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.peft_model(input_ids, attention_mask=attention_mask)  # モデルからの出力
        pooled_output = outputs.last_hidden_state.mean(dim=1)  # 出力の平均値を計算
        output_dropout = self.dropout(pooled_output)  # ドロップアウトを適用
        logits = self.classifier(output_dropout)  # 最終的なロジットを取得
        loss = None
        if labels is not None:
            labels = labels
            loss = loss_fn(logits, labels)  # 損失を計算
        return loss, logits  # 損失とロジットを返す

## パラレルトレーニング


In [ ]:
def parallel_function(model_name, attention_name, file_path):
    mp.set_start_method('spawn', force=True)  # プロセスの開始方法を設定

    accelerator = Accelerator(mixed_precision="fp16")  # 半精度トレーニングを使用
    if accelerator.is_main_process:  # メインプロセスの場合
        datasets.utils.logging.set_verbosity_warning()  # 警告のログレベルを設定
        transformers.utils.logging.set_verbosity_info()  # 情報のログレベルを設定
    else:
        datasets.utils.logging.set_verbosity_error()  # エラーログレベルを設定
        transformers.utils.logging.set_verbosity_error()  # エラーログレベルを設定

    set_seed(seed)  # 乱数シードを設定

    tokenizer = AutoTokenizer.from_pretrained(model_name)  # トークナイザを読み込む
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # パディングトークンを設定
    tokenizer.padding_side = "right"  # fp16トレーニングの問題を修正
    
    model = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16)  # モデルを読み込み
    model = quantize_model(model)  # モデルを量子化
    for idx, layer in enumerate(model.layers):
        replace_attention_module(model.config, layer, idx)  # アテンションモジュールを置き換え
    model = LoraModelForClassification(model)  # LoRAモデルを初期化

    # 学習可能なパラメータの設定
    for param in model.peft_model.parameters():
        param.requires_grad = False
    for param in model.classifier.parameters():
        param.requires_grad = True

    # LoRAの重みを学習可能にする
    for name, module in model.named_modules():
        if isinstance(module, attention_name):
            module.qkv_lora.lora_a.weight.requires_grad = True  # LoRA A層の重み
            module.qkv_lora.lora_b.weight.requires_grad = True  # LoRA B層の重み
            module.o_lora.lora_a.weight.requires_grad = True  # 出力LoRA A層の重み
            module.o_lora.lora_b.weight.requires_grad = True  # 出力LoRA B層の重み

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"総学習可能パラメータ数: {total_params}")  # 学習可能パラメータ数を表示

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)  # オプティマイザの初期化

    # データを前処理し、データローダーを作成
    t_dat, v_dat = preprocess_data(file_path, sample=True)
    train_dataloader = create_dataloaders(t_dat, tokenizer, max_len, batch_size=batch_size, shuffle=True)
    eval_dataloader = create_dataloaders(v_dat, tokenizer, max_len, batch_size=batch_size, shuffle=True)

    # モデルとオプティマイザを準備
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader)

    lr_scheduler = get_linear_schedule_with_warmup(  # 学習率スケジューラを設定
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * n_epoch,
    )

    progress_bar = tqdm(range(n_epoch * len(train_dataloader)), disable=not accelerator.is_main_process)  # プログレスバーの初期化

    # トレーニングと評価のループ
    train_loss = 0
    valid_loss = 0
    
    for epoch in range(n_epoch):
        model.train()  # モデルをトレーニングモードに設定
        for step, batch in enumerate(train_dataloader):
            loss, _ = model(**batch)  # バッチをモデルに渡して損失を計算
            accelerator.backward(loss)  # バックプロパゲーション

            optimizer.step()  # オプティマイザのステップ
            lr_scheduler.step()  # 学習率スケジューラのステップ
            optimizer.zero_grad()  # 勾配をゼロにリセット
            progress_bar.update(1)  # プログレスバーを更新

            train_loss += loss.item()  # トレーニングロスを累積
            
        all_predictions = []
        all_labels = []
        model.eval()  # モデルを評価モードに設定
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                loss, logits = model(**batch)  # 評価バッチに対する損失とロジットを計算
            predictions = logits.argmax(dim=-1)  # 最大のロジットを予測として抽出
            all_predictions.append(accelerator.gather(predictions))  # 予測を収集
            all_labels.append(accelerator.gather(batch["labels"]))  # ラベルを収集
            
            valid_loss += loss.item()  # 検証ロスを累積

        all_predictions = torch.cat(all_predictions)[:len(eval_dataloader)].cpu()  # すべての予測を結合
        all_labels = torch.cat(all_labels)[:len(eval_dataloader)].cpu()  # すべてのラベルを結合

        acc_metric = accuracy_score(all_labels, all_predictions)  # 精度を計算
        eval_metric = f1_score(all_labels, all_predictions, average="macro")  # F1スコアを計算
        train_loss_avg = train_loss / len(train_dataloader)  # トレーニングロスの平均を計算
        valid_loss_avg = valid_loss / len(eval_dataloader)  # 検証ロスの平均を計算
        
        # 結果を表示
        accelerator.print(f"エポック: {epoch} \n 精度: {acc_metric:.3f} \n F1スコア: {eval_metric:.3f} \n トレーニングロス: {train_loss_avg:.3f} \n 検証ロス: {valid_loss_avg:.3f}")

    model = accelerator.unwrap_model(model)  # モデルのラッピングを解除
    accelerator.wait_for_everyone()  # すべてのプロセスが完了するのを待つ

    # モデルの状態を保存
    if accelerator.is_main_process:
        torch.save(model.state_dict(), model_path)  # モデルの状態を保存

    # 同期完了のメッセージ
    accelerator.wait_for_everyone()

In [ ]:
notebook_launcher(parallel_function, args=(model_name, Phi3Attention, file_path,), num_processes=2)  # パラレルファンクションを起動